In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_13_0", "ga-IE", split="train+validation"
)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_13_0", "ga-IE", split="test")

print(common_voice)

Reading metadata...: 549it [00:00, 176707.31it/s]ples/s]
Generating train split: 549 examples [00:00, 1910.21 examples/s]
Reading metadata...: 506it [00:00, 318284.02it/s] examples/s]
Generating validation split: 506 examples [00:00, 13242.55 examples/s]
Reading metadata...: 511it [00:00, 290143.41it/s]les/s]
Generating test split: 511 examples [00:00, 13719.60 examples/s]
Reading metadata...: 4226it [00:00, 324701.47it/s]les/s]
Generating other split: 4226 examples [00:00, 14859.34 examples/s]
Reading metadata...: 882it [00:00, 307640.43it/s]? examples/s]
Generating invalidated split: 882 examples [00:00, 13799.78 examples/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1055
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 511
    })
})


In [10]:
common_voice = common_voice.select_columns(["audio", "sentence"])

In [12]:
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE

fine_tune_languae = "welsh"
fine_tune_code = TO_LANGUAGE_CODE[fine_tune_languae]

In [13]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language=fine_tune_languae, task="transcribe"
)

preprocessor_config.json: 100%|██████████| 185k/185k [00:00<00:00, 8.46MB/s]
tokenizer_config.json: 100%|██████████| 805/805 [00:00<00:00, 3.89MB/s]
vocab.json: 100%|██████████| 836k/836k [00:00<00:00, 4.38MB/s]
tokenizer.json: 100%|██████████| 2.48M/2.48M [00:00<00:00, 5.27MB/s]
merges.txt: 100%|██████████| 494k/494k [00:00<00:00, 12.9MB/s]
normalizer.json: 100%|██████████| 52.7k/52.7k [00:00<00:00, 43.0MB/s]
added_tokens.json: 100%|██████████| 34.6k/34.6k [00:00<00:00, 29.7MB/s]
special_tokens_map.json: 100%|██████████| 2.08k/2.08k [00:00<00:00, 9.07MB/s]


In [14]:
common_voice["train"].features

{'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [15]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [16]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [22]:
common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1
)

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]


RuntimeError: Decoding 'mp3' files requires system library 'libsndfile'>=1.1.0, You can try to update `soundfile` python library: `pip install "soundfile>=0.12.1"`. 